## Importing Libraries and Dataset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os

if not os.path.isdir('models'): 
    os.mkdir('models')   

In [ ]:
import tensorflow as tf
from keras.models import Sequential  #Sequential Model
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout,Activation  #Different layers of CNN
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint,EarlyStopping

In [ ]:
pip install emnist

In [ ]:
#from emnist import list_datasets

In [ ]:
#list_datasets()

In [ ]:
from emnist import extract_training_samples 
X_train,y_train = extract_training_samples('byclass')

In [ ]:
from emnist import extract_test_samples
X_test,y_test  = extract_test_samples('byclass')

In [ ]:
print("X_train shape: ",X_train.shape)
print("y_train shape: ",y_train.shape)
print("X_test shape: ",X_test.shape)
print("y_test shape: ",y_test.shape)

In [ ]:
#In Training Set there are 6,97,932 images of 28 by 28 pixels
#In test Set there are 1,16,323 images of 28 by 28 pixels

## Visualization

In [ ]:
random_index = np.random.randint(0,697932)

print("\t  Label :",y_train[random_index]) 
plt.imshow(X_train[random_index], cmap='gray');

#by adding ; at the end you don't get that matplotlib line....<matplotlib.image.AxesImage at 0x7fa9f8b04a20>

In [ ]:
random_index = np.random.randint(0,697932)

print("\t  Label :",y_train[random_index]) 
plt.imshow(X_train[random_index], cmap='gray');

In [ ]:
print("\t\t\tTABLE\n")
from IPython.display import Image
Image('/content/Images/Table.png')

In [ ]:
classes = ['0','1','2','3','4','5','6','7','8','9',
             'A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z',
             'a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

In [ ]:
#Checking Training Examples
fig = plt.figure(figsize=(12,7))
for i in range(10):
  plt.subplot( 2, 5 , i+1)            # 2 rows 5 columns                                                                  
  plt.imshow(X_train[i], cmap='gray')  # prints image
  plt.yticks([])  #no x and y ticks
  plt.xticks([])
  index = y_train[i]
  plt.title("Character: " + str(classes[index]))      

## Reshaping and One Hot Encoding

In [ ]:
#We need to reshape our 3D array to 4D array so it can be used with keras
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

In [ ]:
print("X_train shape: ",X_train.shape)
print("X_test shape: ",X_test.shape)

In [ ]:
X_train = X_train/255  #normalize inputs from 0-255 to 0-1
X_test = X_test/255

In [ ]:
from keras.utils import to_categorical
#Keras expects "one hot" encoding in multiclass problems for the labels.

print("Shape of y_train:",y_train.shape)
y_train = to_categorical(y_train)   
print("New Shape of y_train:",y_train.shape)
print("\n")
print("Shape of y_test:",y_test.shape)
y_test = to_categorical(y_test)  
print("New Shape of y_test:",y_test.shape)

## MODEL 1

In [ ]:
# Structure of Model

# Input Vector :  784 pixels (28x28)
# Convolutional Layer 1  :  128 filters
# Convolutional Layer 2  :  256 filters
# Output Layer :  62 classes

In [ ]:
def my_model_2():  #3 Layer CNN
 #Input is 28x28 pixel images with 1 channel -> (28, 28, 1)
  model = Sequential()
  
  # ------ Convolutional Layers -------   ( Feature Extraction )
  
  # 1st Convolutional layer
  model.add(Conv2D(128, (3,3), input_shape = (28,28,1),activation='relu'))  # adds 128 filters with 3x3 window size/kernel size            
  model.add(MaxPooling2D(pool_size=(2,2)))
 
  # 2nd Convolutional layer
  model.add(Conv2D(256, (5,5),activation='relu'))  # adds 256 filters with 3x3 kernel size..Activation function is relu
  model.add(MaxPooling2D(pool_size=(2,2)))  #Performs pooling to reduce dimensionality size
  

  #  ------ Fully Connected Layer ------   ( Classification )

  # 3rd layer (Output Layer)
  model.add(Flatten())  # Flattening the matrix into 1D array and feed it into Dense layer.
  model.add(Dense(62,activation='softmax'))
  #ouput layer has 62 neurons because there are 62 classes
  #softmax function is used for multi-class classification

  return model
 


In [ ]:
model1 = my_model_2()
model1.summary()

In [ ]:
model1.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

In [ ]:
# Fit model on training data
history = model1.fit(X_train, y_train,
                  batch_size = 128,
                  epochs = 30,
                  validation_data=(X_test,y_test), 
                  #our model dosen't see this validation data...so it does not get trained on it...
                  #it's only used for tuning hyperparameters...
                  verbose=1,
                  callbacks=[
                 EarlyStopping(monitor='val_accuracy', patience=7)
                # ModelCheckpoint('models/model_{val_accuracy:.3f}.h5', #Models stored in file
                                                 #   save_best_only=True,
                                                  # save_weights_only=False, 
                                                   #monitor='val_accuracy')]
                  ]
             )

In [ ]:
# Evaluate model on test data
loss, accuracy = model1.evaluate(X_test, y_test, verbose=1)
print ('Test loss :', loss )
print ('Test accuracy :', accuracy )

In [ ]:
loss1, accuracy1 = model1.evaluate(X_train, y_train, verbose=1)
print ('Train loss :', loss1 )
print ('Train accuracy :', accuracy1 )

In [ ]:
import seaborn as sns
s = len(history.history['accuracy'])

plt.figsize=(10,10)
sns.lineplot(x = range(1,1+s),y = history.history['accuracy'], label='Accuracy')
sns.lineplot(x = range(1,1+s),y = history.history['val_accuracy'], label='Val_Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

In [ ]:
s = len(history.history['loss'])

plt.figsize=(10,10)
sns.lineplot(x = range(1,1+s),y = history.history['loss'], label='Loss')
sns.lineplot(x = range(1,1+s),y = history.history['val_loss'], label='Val_Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

## MODEL 2

In [ ]:
#  Model Structure
# Input vector (28,28,1)
# 1 hidden layer 128 units
# 1 output Layer with 62 classes

In [ ]:
def my_model_1(inp_shape,output_size):  # 3 layer CNN
  model = Sequential() 
  #Creating Sequential Model

 #Layer 1
  model.add(Conv2D(28, kernel_size=(3,3), input_shape=inp_shape))#This layer is used to extract features from image
  model.add(MaxPooling2D(pool_size=(2, 2)))  # downsampling operation 
  model.add(Flatten()) # Flattening the 2D arrays for fully connected layers into 1D array


  #Layer 2
  model.add(Dense(128, activation=tf.nn.relu))  #Dense layer implies each neuron connected to all other neurons
  #There are 128 neurons here and activation function is relu
  model.add(Dropout(0.2))  #Dropout---> 20% neurons will be dropped....used to avoid overfitting

  #layer 3
  model.add(Dense(output_size,activation=tf.nn.softmax))
  #ouput layer has 62 neurons because there are 62 classes in dataset
  #softmax function is used for multi-class classification
  model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
  return model

In [ ]:
model = my_model_1((28,28,1),62)
model.summary()

In [ ]:
model.fit(X_train,y_train, epochs=8)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
random_index = np.random.randint(0,116323)
plt.imshow(X_test[random_index].reshape(28, 28),cmap='gray')
predictions = model.predict(X_test[random_index].reshape(1, 28, 28, 1))
print("Predicted Label : ",predictions.argmax())

## MODEL 3  ( Good Results )


In [ ]:
# ----- Model Stucture -----

# Input - (28x28) pixel image
# 1st Convolutional Layer - 128 filters
# 2nd Convolutional Layer - 256 filters
# 3rd Output Layer - 62 classes

In [ ]:
def my_model_3():  
# Input: 28x28 images with 1 channel -> (28, 28, 1)
  model = Sequential()
  
  # ----- Feature Extraction ----- ( Convolutional layers )

  # 1st Convolutional layer
  model.add(Conv2D(128, (3,3),activation='relu', input_shape = (28,28,1))) # adds 128 neurons with 3x3 window size/kernel size 
  model.add(BatchNormalization()) 
  model.add(MaxPooling2D(pool_size=(2,2)))

  # 2nd Convolutional layer
  model.add(Conv2D(256, (5,5),activation='relu'))      # 256 filters with 3x3 kernel size..
  model.add(BatchNormalization())                  #Activation function is relu
  model.add(MaxPooling2D(pool_size=(2,2)))  #Perform pooling to reduce dimensionality size

 
  # ----- Classification -----  ( Fully Connected Layer )

  # 3rd layer(Output Layer)
  model.add(Flatten())  # Flattening the matrix into 1D array and feed it into a fully connected layer(Dense layer).

  model.add(Dense(128,activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.3))

  model.add(Dense(62,activation='softmax'))
  #ouput layer has 62 neurons because there are 62 classes
  #softmax function is used for multi-class classification

  return model

In [ ]:
model2 = my_model_3()
model2.summary()

In [ ]:
model2.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

In [ ]:
# Fit model on training data
history = model2.fit(X_train, y_train,
                  batch_size = 128,
                  epochs = 30,
                  validation_data=(X_test,y_test), 
                  #our model dosen't see this validation data...so it does not get trained on it...
                  #it's only used for tuning hyperparameters...
                  verbose=1,
                  callbacks=[
                 EarlyStopping(monitor='val_accuracy', patience=3)
              #   ModelCheckpoint('models/model_{val_accuracy:.3f}.h5', #Models stored in file
                                                  #  save_best_only=True,
                                                   #save_weights_only=False, 
                                                   #monitor='val_accuracy')]
                  ]
             )

### **Evaluate on Model**

In [ ]:
loss, accuracy = model2.evaluate(X_test, y_test, verbose=0)
print ('Test loss :', loss )
print ('Test accuracy :', accuracy )

In [ ]:
loss1, accuracy1 = model2.evaluate(X_train, y_train, verbose=0)
print ('Train loss :', loss1 )
print ('Train accuracy :', accuracy1 )

In [ ]:
import seaborn as sns
s = len(history.history['accuracy'])

plt.figsize=(10,10)
sns.lineplot(x = range(1,1+s),y = history.history['accuracy'], label='Accuracy')
sns.lineplot(x = range(1,1+s),y = history.history['val_accuracy'], label='Val_Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

In [ ]:
s = len(history.history['loss'])

plt.figsize=(10,10)
sns.lineplot(x = range(1,1+s),y = history.history['loss'], label='Loss')
sns.lineplot(x = range(1,1+s),y = history.history['val_loss'], label='Val_Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

### **Predictions**

In [ ]:
random_index = np.random.randint(0,116323)
plt.imshow(X_test[random_index].reshape(28, 28),cmap='gray')
predictions = model1.predict(X_test[random_index].reshape(1, 28, 28, 1))
index1 =predictions.argmax()
print("Predicted Label : "+classes[index1])
index = y_test[random_index].argmax()
print("Actual Label:"+ classes[index])

In [ ]:
random_index = np.random.randint(0,116323)
plt.imshow(X_test[random_index].reshape(28, 28),cmap='gray')
predictions = model1.predict(X_test[random_index].reshape(1, 28, 28, 1))
index1 =predictions.argmax()
print("Predicted Label : "+classes[index1])
index = y_test[random_index].argmax()
print("Actual Label:"+ classes[index])

In [ ]:
random_index = np.random.randint(0,116323)
plt.imshow(X_test[random_index].reshape(28, 28),cmap='gray')
predictions = model1.predict(X_test[random_index].reshape(1, 28, 28, 1))
index1 =predictions.argmax()
print("Predicted Label : "+classes[index1])
index = y_test[random_index].argmax()
print("Actual Label:"+ classes[index])

In [ ]:
plt.imshow(X_test[107596].reshape(28, 28),cmap='gray')
predictions = model1.predict(X_test[107596].reshape(1, 28, 28, 1))
index1 =predictions.argmax()
index = y_test[107596].argmax()
print("Predicted Label : "+classes[index1])

print("Actual Label: "+ classes[index])
if classes[index] != classes[index1]:
  print("\n\tIt can be either 0 or O or o")
  

ans = int(input("Choose :\n1. 0\n2. o\n3. O\n"))

if(ans == 1):
  answer = "0"
elif(ans == 2):
  answer="o"
else:
  answer="O"

print("Your chosen ans is:"+answer)

In [ ]:
fig = plt.figure(figsize=(10,17))
for i in range(12):
  plt.subplot(4 , 3 , i+1)            # 2 rows 5 columns                                                                  
  plt.imshow(X_test[i].reshape(28,28), cmap='gray')  # prints image
  predictions = model1.predict(X_test[i].reshape(1, 28, 28, 1))
  index1 =predictions.argmax()
  index2 = y_test[i].argmax()
  plt.title("Predicted Character : "+classes[index1]
            +"\n"+"Actual Character: "+ classes[index2])
  


**Reference Links**  
[Model created using the steps mentioned in summary and as shown in image.](https://medium.com/@RaghavPrabhu/understanding-of-convolutional-neural-network-cnn-deep-learning-99760835f148)     
[Tensorflow link](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D)

In [ ]:
from IPython.display import Image
Image('/content/Images/Image1.png')

## MODEL 4

In [ ]:
def my_model_4():  
# Input: 28x28 images with 1 channel -> (28, 28, 1)
  model = Sequential()
  
  # ----- Feature Extraction ----- ( Convolutional layers )

  # 1st Convolutional layer
  model.add(Conv2D(128, (3,3), input_shape = (28,28,1))) # adds 128 neurons with 3x3 window size/kernel size 
  model.add(BatchNormalization()) 
  model.add(Activation('relu'))         
  model.add(MaxPooling2D(pool_size=(2,2)))
 

  # ----- Classification -----  ( Fully Connected Layer )

  # 3rd layer(Output Layer)
  model.add(Flatten())  # Flattening the matrix into 1D array and feed it into a fully connected layer(Dense layer).
  model.add(Dense(62))
  model.add(BatchNormalization())     
  model.add(Activation('softmax'))
  #ouput layer has 62 neurons because there are 62 classes
  #softmax function is used for multi-class classification

  return model

In [ ]:
model4 = my_model_4()
model4.summary()

In [ ]:
model4.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

In [ ]:
# Fit model on training data
history = model4.fit(X_train, y_train,
                  batch_size = 128,
                  epochs = 15,
                  validation_data=(X_test,y_test), 
                  #our model dosen't see this validation data...so it does not get trained on it...
                  #it's only used for tuning hyperparameters...
                  verbose=1,
             )

In [ ]:
loss, accuracy = model4.evaluate(X_test, y_test, verbose=0)
print ('Test loss :', loss )
print ('Test accuracy :', accuracy )

In [ ]:
loss1, accuracy1 = model4.evaluate(X_train, y_train, verbose=0)
print ('Train loss :', loss1 )
print ('Train accuracy :', accuracy1 )

In [ ]:
import seaborn as sns
s = len(history.history['accuracy'])

plt.figsize=(10,10)
sns.lineplot(x = range(1,1+s),y = history.history['accuracy'], label='Accuracy')
sns.lineplot(x = range(1,1+s),y = history.history['val_accuracy'], label='Val_Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

In [ ]:
s = len(history.history['loss'])

plt.figsize=(10,10)
sns.lineplot(x = range(1,1+s),y = history.history['loss'], label='Loss')
sns.lineplot(x = range(1,1+s),y = history.history['val_loss'], label='Val_Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')